<a href="https://colab.research.google.com/github/uhdang/mlim_group_3_final_assignment/blob/main/quang/FinalAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
# from datastreamer import DataStreamerP2V
import matplotlib.pyplot as plt
import seaborn as sns
import random

# Load Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')
baskets = pd.read_parquet("/content/drive/MyDrive/My Drive/baskets.parquet")
coupons = pd.read_parquet("/content/drive/MyDrive/My Drive/coupons.parquet")
# coupon_index = pd.read_parquet("/content/drive/MyDrive/My Drive/coupon_index.parquet")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
baskets

,week,shopper,product,price
0,0,0,71,629
1,0,0,91,605
2,0,0,116,715
3,0,0,123,483
4,0,0,157,592
...,...,...,...,...
68841593,89,99999,143,470
68841594,89,99999,158,566
68841595,89,99999,186,499
68841596,89,99999,204,496


In [5]:
coupons

,week,shopper,product,discount
0,0,0,35,35
1,0,0,193,40
2,0,0,27,30
3,0,0,177,35
4,0,0,5,30
...,...,...,...,...
44999995,89,99999,62,40
44999996,89,99999,110,10
44999997,89,99999,37,40
44999998,89,99999,155,25


In [6]:
#baskets[['week', 'shopper', 'product']] = baskets[['week', 'shopper', 'product']].astype('category')
# baskets[['shopper', 'product']] = baskets[['shopper', 'product']].astype('category')
baskets['product'] = baskets['product'].astype('category')
coupons['product'] = coupons['product'].astype('category')

# Split Data

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
weeks = 85 
shoppers = 10

baskets_train = baskets.loc[(baskets.shopper.isin(list(range(shoppers)))) & (baskets.week<weeks), :]
baskets_test = baskets.loc[(baskets.shopper.isin(list(range(shoppers)))) & (baskets.week>=weeks), :]

coupons_train = coupons.loc[(coupons.shopper.isin(list(range(shoppers)))) & (coupons.week<weeks), :]
coupons_test = coupons.loc[(coupons.shopper.isin(list(range(shoppers)))) & (coupons.week>=weeks), :]

In [9]:
baskets_train['shopper'] = baskets_train['shopper'].astype('category')
coupons_train['shopper'] = coupons_train['shopper'].astype('category')

baskets_train['target'] = '1'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [10]:
baskets_train

,week,shopper,product,price,target
0,0,0,71,629,1
1,0,0,91,605,1
2,0,0,116,715,1
3,0,0,123,483,1
4,0,0,157,592,1
...,...,...,...,...,...
64253759,84,9,71,629,1
64253760,84,9,86,481,1
64253761,84,9,168,588,1
64253762,84,9,192,549,1


In [11]:
coupons_train

,week,shopper,product,discount
0,0,0,35,35
1,0,0,193,40
2,0,0,27,30
3,0,0,177,35
4,0,0,5,30
...,...,...,...,...
42000045,84,9,24,10
42000046,84,9,56,25
42000047,84,9,31,30
42000048,84,9,185,35


# Data Preprocessing

In [12]:
full_df_train = (baskets_train
           .merge(
               coupons_train, 
               on=['week', 'shopper', 'product'], 
               how='outer',
               indicator=True
           )
           .sort_values(by=['week', 'shopper', 'product'])
           .reset_index(drop=True)
          #  .replace(['left_only', 'right_only', 'both'], ['RRP', 'coupon', 'discounted'])
          .replace(['left_only', 'right_only', 'both'], ['No', 'Yes', 'Yes'])
          .rename(columns={'_merge': 'coupon'})
)

full_df_train.loc[full_df_train['discount'].isna(),'discount' ] = 0

In [13]:
full_df_train

,week,shopper,product,price,target,discount,coupon
0,0,0,5,NaN,NaN,30.0,Yes
1,0,0,27,NaN,NaN,30.0,Yes
2,0,0,35,NaN,NaN,35.0,Yes
3,0,0,71,629.0,1,0.0,No
4,0,0,91,605.0,1,0.0,No
...,...,...,...,...,...,...,...
10753,84,9,168,588.0,1,0.0,No
10754,84,9,169,NaN,NaN,35.0,Yes
10755,84,9,185,NaN,NaN,35.0,Yes
10756,84,9,192,549.0,1,0.0,No


In [14]:
# Counting weeks of prior order for each product/customer
full_df_train['weeks_since_prior_order'] = full_df_train.groupby(['shopper', 'product']).week.apply(lambda x: x.diff()).fillna(0) # change to count weeks since last order, even when it wasn't bought

In [15]:
full_df_train

,week,shopper,product,price,target,discount,coupon,weeks_since_prior_order
0,0,0,5,NaN,NaN,30.0,Yes,0.0
1,0,0,27,NaN,NaN,30.0,Yes,0.0
2,0,0,35,NaN,NaN,35.0,Yes,0.0
3,0,0,71,629.0,1,0.0,No,0.0
4,0,0,91,605.0,1,0.0,No,0.0
...,...,...,...,...,...,...,...,...
10753,84,9,168,588.0,1,0.0,No,5.0
10754,84,9,169,NaN,NaN,35.0,Yes,10.0
10755,84,9,185,NaN,NaN,35.0,Yes,30.0
10756,84,9,192,549.0,1,0.0,No,1.0


In [16]:
# offered_price = 
original_price = baskets.groupby('product', as_index=False).price.max().rename(columns={'price': 'original_price'})
total_count_of_product = baskets_train.groupby(['shopper', 'product']).product.count().to_frame('total_count_of_product').reset_index()
reordered = (baskets_train.groupby(['shopper']).product.value_counts()>1).astype(int)
average_price_per_shopper = baskets_train.groupby(['shopper']).price.mean().to_frame('average_price_per_shopper').reset_index()
average_basket_size = baskets_train.groupby(['shopper', 'week']).product.count().groupby('shopper').mean().to_frame('average_basket_size').reset_index()
unique_products_per_shopper = baskets_train.groupby(['shopper']).product.nunique()
ratio_of_reordered_per_shopper = (reordered.groupby('shopper').sum() / unique_products_per_shopper).to_frame('ratio_of_reordered').reset_index()

ratio_of_reordered_per_product = total_count_of_product.copy()
ratio_of_reordered_per_product['total_count_of_product'] = total_count_of_product['total_count_of_product'] / (baskets_train.week.max()+1)
ratio_of_reordered_per_product.rename(columns={'total_count_of_product': 'ratio_of_reordered_per_product'}, inplace=True)

reordered = reordered.to_frame('reordered').reset_index()
unique_products_per_shopper = unique_products_per_shopper.to_frame('unique_products_per_shopper').reset_index()

# add average interval for each product and shopper
# WoE

In [17]:
# baskets_train = (baskets_train
#                  .merge(total_count_of_product, on=['shopper', 'product'])
#                  .merge(reordered, on=['shopper', 'product'])
#                  .merge(ratio_of_reordered_per_product, on=['shopper', 'product'])
#                  .merge(ratio_of_reordered_per_shopper, on=['shopper'])
#                  .merge(average_price_per_shopper, on=['shopper'])
#                  .merge(average_basket_size, on=['shopper'])
#                  .merge(unique_products_per_shopper, on=['shopper'])
#                  )

# Set Up Training Table

In [18]:
df1 = pd.DataFrame({'key':np.ones(weeks), 'week':list(range(weeks))})
df2 = pd.DataFrame({'key':np.ones(shoppers), 'shopper':list(range(shoppers))})
df3 = pd.DataFrame({'key':np.ones(250), 'product':list(range(250))})
df4 = full_df_train.copy()

X_train = pd.merge(df1, df2, on='key').merge(df3, on='key').merge(df4, on=['week', 'shopper', 'product'], how='left')[full_df_train.columns]

In [19]:
X_train

,week,shopper,product,price,target,discount,coupon,weeks_since_prior_order
0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,0,0,1,NaN,NaN,NaN,NaN,NaN
2,0,0,2,NaN,NaN,NaN,NaN,NaN
3,0,0,3,NaN,NaN,NaN,NaN,NaN
4,0,0,4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
212495,84,9,245,NaN,NaN,NaN,NaN,NaN
212496,84,9,246,NaN,NaN,NaN,NaN,NaN
212497,84,9,247,NaN,NaN,NaN,NaN,NaN
212498,84,9,248,NaN,NaN,NaN,NaN,NaN


Combine everything

In [20]:
X_train = (X_train
           .merge(original_price, on=['product'], how='left')
           .merge(total_count_of_product, on=['shopper', 'product'], how='left')
           .merge(reordered, on=['shopper', 'product'], how='left')
           .merge(ratio_of_reordered_per_product, on=['shopper', 'product'], how='left')
           .merge(ratio_of_reordered_per_shopper, on=['shopper'], how='left')
           .merge(average_price_per_shopper, on=['shopper'], how='left')
           .merge(average_basket_size, on=['shopper'], how='left')
           .merge(unique_products_per_shopper, on=['shopper'], how='left')
           )

In [21]:
X_train

,week,shopper,product,price,target,discount,coupon,weeks_since_prior_order,original_price,total_count_of_product,reordered,ratio_of_reordered_per_product,ratio_of_reordered,average_price_per_shopper,average_basket_size,unique_products_per_shopper
0,0,0,0,NaN,NaN,NaN,NaN,NaN,688,0,NaN,0.000000,0.735849,588.063100,8.576471,53
1,0,0,1,NaN,NaN,NaN,NaN,NaN,560,0,NaN,0.000000,0.735849,588.063100,8.576471,53
2,0,0,2,NaN,NaN,NaN,NaN,NaN,773,0,NaN,0.000000,0.735849,588.063100,8.576471,53
3,0,0,3,NaN,NaN,NaN,NaN,NaN,722,0,NaN,0.000000,0.735849,588.063100,8.576471,53
4,0,0,4,NaN,NaN,NaN,NaN,NaN,620,8,1.0,0.094118,0.735849,588.063100,8.576471,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212495,84,9,245,NaN,NaN,NaN,NaN,NaN,549,1,0.0,0.011765,0.703704,570.611814,8.364706,81
212496,84,9,246,NaN,NaN,NaN,NaN,NaN,702,0,NaN,0.000000,0.703704,570.611814,8.364706,81
212497,84,9,247,NaN,NaN,NaN,NaN,NaN,670,0,NaN,0.000000,0.703704,570.611814,8.364706,81
212498,84,9,248,NaN,NaN,NaN,NaN,NaN,490,0,NaN,0.000000,0.703704,570.611814,8.364706,81


In [22]:
X_train.isnull().any()

week                              False
shopper                           False
product                           False
price                              True
target                             True
discount                           True
coupon                             True
weeks_since_prior_order            True
original_price                    False
total_count_of_product            False
reordered                          True
ratio_of_reordered_per_product    False
ratio_of_reordered                False
average_price_per_shopper         False
average_basket_size               False
unique_products_per_shopper       False
dtype: bool

In [23]:
X_train['discount'].fillna(0, inplace=True)
X_train['price'].fillna(X_train.original_price*(1-X_train.discount), inplace=True)
X_train['reordered'].fillna(0, inplace=True)
X_train['coupon'].fillna('No', inplace=True)
X_train['weeks_since_prior_order'].fillna(0, inplace=True)
X_train['target'].fillna('0', inplace=True)

In [24]:
X_train.drop('week', inplace=True, axis=1)

In [25]:
target = X_train.pop('target')

# One-Hot Encoding Products

In [26]:
from sklearn.preprocessing import OneHotEncoder

In [27]:
enc = OneHotEncoder()

encodings = X_train.select_dtypes(exclude=np.number).columns.tolist()
encoded = enc.fit_transform(X_train[encodings])

In [28]:
X_train = pd.concat([X_train, pd.DataFrame.sparse.from_spmatrix(encoded)], axis=1).drop(encodings, axis=1)

In [29]:
X_train

,price,discount,weeks_since_prior_order,original_price,total_count_of_product,reordered,ratio_of_reordered_per_product,ratio_of_reordered,average_price_per_shopper,average_basket_size,unique_products_per_shopper,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,...,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261
0,688.0,0.0,0.0,688,0,0.0,0.000000,0.735849,588.063100,8.576471,53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,560.0,0.0,0.0,560,0,0.0,0.000000,0.735849,588.063100,8.576471,53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,773.0,0.0,0.0,773,0,0.0,0.000000,0.735849,588.063100,8.576471,53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,722.0,0.0,0.0,722,0,0.0,0.000000,0.735849,588.063100,8.576471,53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,620.0,0.0,0.0,620,8,1.0,0.094118,0.735849,588.063100,8.576471,53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212495,549.0,0.0,0.0,549,1,0.0,0.011765,0.703704,570.611814,8.364706,81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
212496,702.0,0.0,0.0,702,0,0.0,0.000000,0.703704,570.611814,8.364706,81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
212497,670.0,0.0,0.0,670,0,0.0,0.000000,0.703704,570.611814,8.364706,81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
212498,490.0,0.0,0.0,490,0,0.0,0.000000,0.703704,570.611814,8.364706,81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


# Training

In [30]:
import lightgbm as lgb

In [31]:
lgb_clf = lgb.LGBMClassifier()